# Baseline BERT Results

## BERT Setup

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\surfd\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [21]:
import pandas as pd

def map_star_to_sentiment(star):
    if '1 star' in star or '2 stars' in star:
        return "negative"
    elif '3 stars' in star:
        return "neutral"
    else:
        return "positive"

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

## Baseline BERT Results

### 2020 Election Cycle Accuracy

In [1]:
df = pd.read_csv('./data/labeled_comments.csv', encoding='utf-8')

df['star_level'] = df['text'].apply(lambda x: classifier(x, **tokenizer_kwargs)[0]['label'])

df['baseline'] = df['star_level'].apply(map_star_to_sentiment)

NameError: name 'pd' is not defined

In [22]:
df.head(5)

,text,created_utc,ups,subreddit,neg,neu,pos,compound,sentiment,star_level,baseline
0,"Very true, but the problem is that even at tha...",1590975768,1,democrats,0.239,0.645,0.116,-0.9769,negative,3 stars,neutral
1,You’re full of crap.\n\nYou present no facts w...,1590984758,2,democrats,0.163,0.692,0.145,-0.1101,negative,1 star,negative
2,"Politically speaking, there is absolutely no w...",1590995791,29,democrats,0.062,0.823,0.115,0.9925,positive,2 stars,negative
3,This is what happens when the president of the...,1591008451,1,democrats,0.000,0.903,0.097,0.4215,positive,1 star,negative
4,Silly comment. No matter how bad a president h...,1591013669,1,democrats,0.261,0.647,0.092,-0.8060,negative,3 stars,neutral


In [23]:
accuracy = (df['sentiment'] == df['baseline']).mean()

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 51.52%


### 2024 Election Cycle Accuracy

In [ ]:
df = pd.read_csv('./data/labeled_comments_2024.csv', encoding='utf-8')

df['star_level'] = df['text'].apply(lambda x: classifier(x, **tokenizer_kwargs)[0]['label'])

df['baseline'] = df['star_level'].apply(map_star_to_sentiment)

In [ ]:
accuracy = (df['sentiment'] == df['baseline']).mean()

print(f"Accuracy: {accuracy * 100:.2f}%")